# 列舉所有資產的riskiness r

In [1]:
import sympy
from pandas import Series,DataFrame
from scipy.optimize import fsolve
import math
import time
import pandas as pd
import numpy  as  np
import os
from datetime import datetime
from dateutil.relativedelta import relativedelta

# 無風險利率
* 我們須扣除無風險利率，得到超額報酬率以計算Riskiness r
* 目前使用LIBOR [1個月利率](https://www.global-rates.com/interest-rates/libor/american-dollar/usd-libor-interest-rate-overnight.aspx)

In [2]:
rfrate = 2.45785 / 12 /100
rfrate

0.0020482083333333333

# 自定義 functions，用來計算風險指標
- get_riskiness_r(guess , arr_returns)
    - 計算Riskiness R
- get_all_risk_with_return_data(df)
    - 傳入多組ETF的月報酬序列資料，分別計算各組ETF的Riskiness R
- get_top5(df)
    - 依照R值排序，並傳回前5名的ETF
    
### 關於超額報酬的數量級:
* 將要帶入公式的數值調整到介在0-100間，目的是為了讓大部分的Riskiness r介在0-100間。

In [3]:
def f1(x,arr_returns):
    return sum( np.exp(-1*arr_returns/x) ) - len(arr_returns)

def get_riskiness_r(guess,arr_returns):
    while (guess<1000):
        risk2 = fsolve(f1,guess,arr_returns)  #引用函數f1
        if risk2[0] != guess:
            break
        guess = guess*10
    return risk2[0]

def get_all_risk_with_return_data(df):
    global rfrate
    
    etf_list = list(df.columns)
    etf_risk = []
    
    # 計算每支ETF的riskiness r
    for etf_name in etf_list:
        etf_returns = df[etf_name] - rfrate  
        guess = 10**(-5)
        risk_r = get_riskiness_r(guess,etf_returns)
        etf_risk.append([etf_name,risk_r])
        
    df = pd.DataFrame(etf_risk,columns=['Name','Riskiness R'])
    return df

def get_top5(df):
    df_sorted = df.sort_values(by=['Riskiness R'])
    df_top5 = df_sorted[:5]
    return df_top5

# 自定義 functions，用來計算最佳配置的權重值


In [4]:
def risk(x):
    guess  = 10**(-5)
    while (guess<1000):
        risk2 = fsolve(f1,guess,x)  #引用函數f1
        if risk2[0] != guess:
            break
        guess = guess*10
    return risk2[0]
    #return np.std(x)

def sumrisk(weight,all_return):
    w = weight
    a = all_return
    v = 0
    for i in range(len(w)):
        v = v + w[i]*risk(w[i]*(all_return.iloc[:,i]))
    return v

def penalty(w):
    if min(w)<0 or max(w)>1:
        return 100000
    else:
        return 0

def portretmean(weight,all_return):
    w = weight
    a = all_return
    return np.mean(np.dot(np.array(all_return),np.array(weight)))    

def bestweight(w,all_return):
    r = []
    r.append(penalty(w)+np.abs(sum(w)-1))
    for i in range(len(w)-1):
        v =  np.abs( risk(w[i]*(all_return.iloc[:,i]))/ sumrisk(w,all_return)  \
            - (np.mean(all_return.iloc[:,i])-rfrate)/(portretmean(w,all_return)-rfrate) ) \
            + np.abs( risk(w[i+1]*(all_return.iloc[:,i+1]))/ sumrisk(w,all_return)  \
            - (np.mean(all_return.iloc[:,i+1])-rfrate)/(portretmean(w,all_return)-rfrate))
        r.append(v) 
    print('權重:',w,'誤差值:',v+penalty(w)+np.abs(sum(w)-1))
    return r

def fourdrop(x):
    a = []
    for i in range(len(x)):
        a.append(round(sum(x[i]),3))
    return a

# 完整的處理流程

依序產生以下檔案

- 每月資料夾下有4個檔案
    - all_group_return.csv：用來存所有群組的月報酬(該月報酬即為該群前5名ETF的月報酬之平均值)
    - all_group_risk.csv：用來存每個群組的Riskiness R(該Riskiness R是以該群月報酬去計算)
    - top5_group_list.csv：用來存所有群組中Riskiness R最小的前5名
    - top5_group_weight.csv：用來存前5名群組的最佳配置權重
    
- 每群資料夾下也有4個檔案
    - all_etf_return.csv：用來存該群中所有ETF的月報酬資料
    - all_etf_risk.csv：用來存該群中每支ETF以月報酬計算出來的Riskiness R
    - top5_etf_list.csv：用來存該群中前5名的ETF(以 Riskiness R 排序)
    - top5_etf_return.csv：用來存該群中前5名的ETF的月報酬取之平均值

In [5]:
def get_total_months(data_path):
    subdir = os.listdir(data_path)[0] #取第一個group的csv檔
    file_path = data_path + subdir + '/' + subdir + '_return.csv'
    if os.path.isfile(file_path):
        df = pd.read_csv(file_path)
        total_rows = df.shape[0]
        return total_rows

In [6]:
return_data_path = '../2014to2018RE_data/'
risk_data_path = '../2017to2019Risk_data/'

# 以程式取得資料總筆數
total_data = get_total_months(return_data_path)
print('共有',total_data,'個月的資料')

# 指定用來計算指標的資料個數
window_size = 36
print('取',window_size,'個月的資料計算風險指標')

rolling_times = total_data - window_size
print('共滾動',rolling_times,'次')

共有 61 個月的資料
取 36 個月的資料計算風險指標
共滾動 25 次


In [7]:
for i in range(rolling_times):
    print('=== Rolling #',i+1,'===')
    
    #第一層目錄
    first_month = '2017-01-01'
    first_date = datetime.strptime(first_month,'%Y-%m-%d')
    the_date = first_date + relativedelta(months=i)
    date_string = the_date.strftime('%Y-%m-%d')
    risk_dir_path = os.path.join(risk_data_path,date_string)
    if not os.path.exists(risk_dir_path):
        os.makedirs(risk_dir_path)
        print('新增資料夾',risk_dir_path)
        
    #第二層目錄
    risk_dir_path_2nd = os.path.join(risk_dir_path,'all_groups/')
    if not os.path.exists(risk_dir_path_2nd):
        os.makedirs(risk_dir_path_2nd)
        print('新增資料夾',risk_dir_path_2nd)

    allgroup_top5_return = DataFrame()
    allgroup_top5_dict = dict()
    
    for subdir in os.listdir(return_data_path):
        #第三層目錄(group name)
        risk_dir_path_3rd = os.path.join(risk_dir_path_2nd,subdir)
        if not os.path.exists(risk_dir_path_3rd):
            os.makedirs(risk_dir_path_3rd)
            print('新增資料夾',risk_dir_path_3rd)
        
        subdir_path = os.path.join(return_data_path,subdir)
        if os.path.exists(subdir_path) and os.path.isdir(subdir_path):
            file = subdir + '_return.csv'
            file_path = os.path.join(subdir_path, file)
            if os.path.isfile(file_path):
                df = pd.read_csv(file_path)
                df_return = df.iloc[i:i+window_size,2:]
                print(df_return.shape)
                df_return = df_return.dropna(axis=1)
                print(df_return.shape)
                
                #第三層目錄下的檔案
                # 0. all_etf_return.csv
                csv_file = "all_etf_return.csv"
                save_path = os.path.join(risk_dir_path_3rd,csv_file)
                df_return.to_csv(save_path,index=False)
                
                # 1. all_etf_risk.csv
                df_etf_risk = get_all_risk_with_return_data(df_return)
                csv_file = "all_etf_risk.csv"
                save_path = os.path.join(risk_dir_path_3rd,csv_file)
                df_etf_risk.to_csv(save_path,index=False)
                
                # 2. top5_etf_list.csv
                df_top5_etf = get_top5(df_etf_risk)
                csv_file = "top5_etf_list.csv"
                save_path = os.path.join(risk_dir_path_3rd,csv_file)
                df_top5_etf.to_csv(save_path,index=False)
                
                # 3. top5_etf_return.csv
                top5_etf = df_top5_etf['Name']
                top5_return_average = df_return[top5_etf].mean(axis=1)
                allgroup_top5_return[subdir] = top5_return_average
                
                csv_file = "top5_etf_return.csv"
                save_path = os.path.join(risk_dir_path_3rd,csv_file)
                top5_return_average.to_csv(save_path,index=False)
                
                # 4. the_group_risk.csv
                # 改在第二層
                
                #[ top5_etf,allgroup_top5_return[subdir] ] = get_top5_etf(df)
                print('Group:',subdir)
                print('Top5:',top5_etf)
                print('================')
                #allgroup_top5_dict[subdir] = top5_etf
                
    #第二層
    # 4. all_group_return
    csv_file = "all_group_return.csv"
    save_path = os.path.join(risk_dir_path,csv_file)
    allgroup_top5_return.to_csv(save_path,index=False)
    
    # 5. all_group_risk
    df_group_risk = get_all_risk_with_return_data(allgroup_top5_return)
    csv_file = "all_group_risk.csv"
    save_path = os.path.join(risk_dir_path,csv_file)
    df_group_risk.to_csv(save_path,index=False)
    
    # 6. top5_group_list
    df_top5_group = get_top5(df_group_risk)
    csv_file = "top5_group_list.csv"
    save_path = os.path.join(risk_dir_path,csv_file)
    df_top5_group.to_csv(save_path,index=False)
    
    # 7. top_group_weight
    top5_group = df_top5_group['Name']    
    top5_group_return = allgroup_top5_return[top5_group]
    n = top5_group_return.shape[1]
    weight = [1/n]*n
    top5_group_best_weight = fsolve(bestweight,weight,top5_group_return)
    print(top5_group_best_weight)
    
    weight_list = []
    for i,group_name in enumerate(top5_group):
        group_weight = top5_group_best_weight[i]
        weight_list.append([group_name,group_weight])
    
    df_best_weight = pd.DataFrame(weight_list,columns=['Name','Weight'])
    print(df_best_weight)
    
    csv_file = "top5_group_weight.csv"
    save_path = os.path.join(risk_dir_path,csv_file)
    df_best_weight.to_csv(save_path,index=False)


=== Rolling # 1 ===
新增資料夾 ../2017to2019Risk_data/2017-01-01
新增資料夾 ../2017to2019Risk_data/2017-01-01/all_groups/
新增資料夾 ../2017to2019Risk_data/2017-01-01/all_groups/Alternatives ETF List (35)
(36, 21)
(36, 14)


/Users/alexis/anaconda3/lib/python3.7/site-packages/scipy/optimize/minpack.py:163: RuntimeWarning: The iteration is not making good progress, as measured by the 
  improvement from the last ten iterations.
  warnings.warn(msg, RuntimeWarning)
/Users/alexis/anaconda3/lib/python3.7/site-packages/scipy/optimize/minpack.py:163: RuntimeWarning: The number of calls to function has reached maxfev = 400.
  warnings.warn(msg, RuntimeWarning)


Group: Alternatives ETF List (35)
Top5: 3     FLAG
7      MOM
12     SIZ
1     CHEP
13    WTMF
Name: Name, dtype: object
新增資料夾 ../2017to2019Risk_data/2017-01-01/all_groups/Healthcare Equity ETF List (49)
(36, 34)
(36, 25)
Group: Healthcare Equity ETF List (49)
Top5: 24    XPH
1     BIB
9     IHI
21    XHE
8     IHF
Name: Name, dtype: object
新增資料夾 ../2017to2019Risk_data/2017-01-01/all_groups/Developed Markets ETF List 1 (100)
(36, 80)
(36, 66)
Group: Developed Markets ETF List 1 (100)
Top5: 26    IDOG
62     VSS
63      VT
59    URTH
39     KXI
Name: Name, dtype: object
新增資料夾 ../2017to2019Risk_data/2017-01-01/all_groups/Target Maturity Date Corporate Bond ETF List (24)
(36, 16)
(36, 6)
Group: Target Maturity Date Corporate Bond ETF List (24)
Top5: 4    IBDC
2    IBCD
3    IBCE
1    BSCM
0    BSCJ
Name: Name, dtype: object
新增資料夾 ../2017to2019Risk_data/2017-01-01/all_groups/Global ETF List (72)
(36, 40)
(36, 31)
Group: Global ETF List (72)
Top5: 0     AADR
15    GDXJ
13    FONE
19     IFV

權重: [0.2 0.2 0.2 0.2 0.2] 誤差值: 3.0043742051271325
權重: [0.2 0.2 0.2 0.2 0.2] 誤差值: 3.004374196727471
權重: [0.2 0.2 0.2 0.2 0.2] 誤差值: 3.0043741705078775
權重: [0.2 0.2 0.2 0.2 0.2] 誤差值: 3.004374184524064
權重: [ 22.55694245   0.32513842  24.65295858 -28.30046166 -17.62199571] 誤差值: 100001.97896580203
權重: [ 0.10498541  0.39999792 -6.98635033 -1.40481584  6.19420281] 誤差值: 100004.90598738665
權重: [ 0.52635872  0.39999775 -3.40374356  0.70377581  0.1339991 ] 誤差值: 100004.14178326054
權重: [0.22283099 0.40000194 0.97283226 0.24700374 0.31465185] 誤差值: 2.6558672526983607
權重: [-0.0610177   0.45558722  0.79336532  0.62703177  0.61387107] 誤差值: 100002.16740869291
權重: [0.26940127 0.59812823 1.71908814 0.25809327 0.43469264] 誤差值: 100003.20494615352
權重: [0.222831   0.40000194 0.97283226 0.24700374 0.31465185] 誤差值: 2.6558672422011713
權重: [0.22283099 0.40000194 0.97283226 0.24700374 0.31465185] 誤差值: 2.655867250263359
權重: [0.22283099 0.40000194 0.97283227 0.24700374 0.31465185] 誤差值: 2.6558672598313695
權重: [0.222830

Group: Consumer Discretionary Equity ETF List (31)
Top5: 19     XLY
11    PSCD
15     RXI
3     FDIS
18     VCR
Name: Name, dtype: object
新增資料夾 ../2017to2019Risk_data/2017-02-01/all_groups/Total Bond Market ETF List (82)
(36, 36)
(36, 23)
Group: Total Bond Market ETF List (82)
Top5: 10    HOLD
16    NEAR
21    ULST
15    MINT
22    XMPT
Name: Name, dtype: object
新增資料夾 ../2017to2019Risk_data/2017-02-01/all_groups/Developed Markets ETF List 3 (93)
(36, 28)
(36, 15)
Group: Developed Markets ETF List 3 (93)
Top5: 1     CROP
13     PEX
7     FTAG
11     IPO
6      FLM
Name: Name, dtype: object
新增資料夾 ../2017to2019Risk_data/2017-02-01/all_groups/Treasuries ETF List (51)
(36, 41)
(36, 38)
Group: Treasuries ETF List (51)
Top5: 27     TUZ
17     SHY
36    VGSH
14    SCHO
20    STPP
Name: Name, dtype: object
新增資料夾 ../2017to2019Risk_data/2017-02-01/all_groups/Gold ETF List (17)
(36, 12)
(36, 11)
Group: Gold ETF List (17)
Top5: 1     DGLD
10    UGLD
3      DGZ
7      IAU
9      UBG
Name: Name, dtyp

...

權重: [0.29152431 0.21239159 0.10620162 0.18190316 0.26462784] 誤差值: 0.32091037247630627
權重: [0.33580997 0.23360886 0.10294825 0.1862969  0.2671661 ] 誤差值: 0.332363967787487
權重: [0.32104809 0.22653643 0.1040327  0.18483232 0.26632001] 誤差值: 0.33233176027509925
權重: [0.32104809 0.22653644 0.1040327  0.18483232 0.26632001] 誤差值: 0.33233175857559294
權重: [0.32104809 0.22653643 0.10403271 0.18483232 0.26632001] 誤差值: 0.3323317578093372
權重: [0.32104809 0.22653643 0.1040327  0.18483232 0.26632001] 誤差值: 0.33233177616658494
權重: [0.32104809 0.22653643 0.1040327  0.18483232 0.26632002] 誤差值: 0.3323317371713699
權重: [0.31274003 0.22443307 0.10202238 0.18339092 0.26837516] 誤差值: 0.2823013677295756
權重: [0.31909739 0.22756206 0.10588818 0.18555844 0.26661834] 誤差值: 0.33596904372674596
權重: [0.32404072 0.2268204  0.10403127 0.18501966 0.26563716] 誤差值: 0.33922087754379415
權重: [0.31955177 0.22639445 0.10403342 0.18473865 0.26666144] 誤差值: 0.328890779261374
權重: [0.316569   0.22610678 0.10403901 0.18455154 0.26734584] 

Group: Preferred Stock ETF List (12)
Top5: 5     PSK
4     PGF
3    PFXF
2     PFF
0     FPE
Name: Name, dtype: object
新增資料夾 ../2017to2019Risk_data/2018-12-01/all_groups/Emerging Asia Pacific ETF List (114)
(36, 78)
(36, 75)
Group: Emerging Asia Pacific ETF List (114)
Top5: 67     VWO
1     ADRE
10    CHAU
73    YINN
69    XINA
Name: Name, dtype: object
新增資料夾 ../2017to2019Risk_data/2018-12-01/all_groups/Energy Equity ETF List (80)
(36, 60)
(36, 59)
Group: Energy Equity ETF List (80)
Top5: 23    GUSH
19    GASL
8     DRIP
5     CRAK
47     PZD
Name: Name, dtype: object
新增資料夾 ../2017to2019Risk_data/2018-12-01/all_groups/Emerging Markets ETF List (79)
(36, 44)
(36, 44)
Group: Emerging Markets ETF List (79)
Top5: 2     BICK
13     EEB
8     DVYE
0     ADRE
40    TLEH
Name: Name, dtype: object
新增資料夾 ../2017to2019Risk_data/2018-12-01/all_groups/Telecom Equity ETF List (10)
(36, 8)
(36, 8)
Group: Telecom Equity ETF List (10)
Top5: 4     IYZ
1    FCOM
6     VOX
7     XTL
2    FONE
Name: Name, 

權重: [0.4144944  0.17410017 0.17250149 0.23201371 0.18221935] 誤差值: 0.35264929039609205
權重: [0.4144944  0.17410017 0.17250149 0.2320137  0.18221936] 誤差值: 0.352649328320807
權重: [0.40676666 0.17176916 0.17152872 0.2345981  0.18109602] 誤差值: 0.32277779447839294
權重: [0.39078352 0.16674011 0.1702306  0.23957218 0.17930383] 誤差值: 0.26714409618663515
權重: [0.41475824 0.17428368 0.17217778 0.23211107 0.18199212] 誤差值: 0.35065241150038307
權重: [0.40676667 0.17176916 0.17152872 0.2345981  0.18109602] 誤差值: 0.3227777984362954
權重: [0.40676666 0.17176916 0.17152872 0.2345981  0.18109602] 誤差值: 0.32277779638873105
權重: [0.40676666 0.17176916 0.17152873 0.2345981  0.18109602] 誤差值: 0.32277779649373484
權重: [0.40676666 0.17176916 0.17152872 0.23459811 0.18109602] 誤差值: 0.3227777743567004
權重: [0.40676666 0.17176916 0.17152872 0.2345981  0.18109603] 誤差值: 0.32277781236024805
權重: [0.41017201 0.17320784 0.17097192 0.23554669 0.18049812] 誤差值: 0.316274463954501
權重: [0.41695164 0.17617263 0.16995598 0.23736168 0.17936699]

Group: Consumer Staples Equity ETF List (26)
Top5: 14    WBIE
8      PSL
7     PSCC
12     VDC
1     EMCG
Name: Name, dtype: object
新增資料夾 ../2017to2019Risk_data/2019-01-01/all_groups/Developed Asia Pacific ETF List (108)
(36, 61)
(36, 60)
Group: Developed Asia Pacific ETF List (108)
Top5: 6      DOL
49    JPMV
8      DTH
41    IDOG
7      DOO
Name: Name, dtype: object
新增資料夾 ../2017to2019Risk_data/2019-01-01/all_groups/Materials Equity ETF List (62)
(36, 52)
(36, 52)
Group: Materials Equity ETF List (62)
Top5: 16    GUNR
15    GRES
22    JDST
4     DUST
11    GDXS
Name: Name, dtype: object
新增資料夾 ../2017to2019Risk_data/2019-01-01/all_groups/Real Estate ETF List (48)
(36, 35)
(36, 35)
Group: Real Estate ETF List (48)
Top5: 24     RWX
10    IFGL
3      FFR
33     WPS
20     REM
Name: Name, dtype: object
新增資料夾 ../2017to2019Risk_data/2019-01-01/all_groups/Utilities Equity ETF List (24)
(36, 17)
(36, 17)
Group: Utilities Equity ETF List (24)
Top5: 2      FXU
4      GII
8      JXI
15    WBIC
1

權重: [0.14241693 0.06005754 0.12362436 0.39672138 0.37014563] 誤差值: 0.6091604819532989
權重: [0.14241693 0.06005754 0.12362435 0.39672139 0.37014563] 誤差值: 0.6091605003217034
權重: [0.14241693 0.06005754 0.12362435 0.39672138 0.37014563] 誤差值: 0.609160461844333
權重: [0.14153364 0.0595588  0.11685052 0.39650504 0.37155847] 誤差值: 0.5862242373488051
權重: [0.14285858 0.06030692 0.12701127 0.39682955 0.3694392 ] 誤差值: 0.6142210476375792
權重: [0.14219611 0.05993286 0.1219309  0.39666729 0.37049884] 誤差值: 0.6066344614567403
權重: [0.14177814 0.05970058 0.11855233 0.39655737 0.37121263] 誤差值: 0.58846652420856
權重: [0.14240509 0.060049   0.12362018 0.39672226 0.37014194] 誤差值: 0.6091615176423493
權重: [0.14219611 0.05993286 0.1219309  0.39666729 0.37049884] 誤差值: 0.6066344630954821
權重: [0.14219611 0.05993286 0.1219309  0.39666729 0.37049884] 誤差值: 0.6066344617912852
權重: [0.14219611 0.05993286 0.1219309  0.39666729 0.37049884] 誤差值: 0.6066344624269154
權重: [0.14219611 0.05993286 0.1219309  0.3966673  0.37049884] 誤差值: 0.